<a href="https://www.kaggle.com/code/andrey36912/homework-mood-3-part-5?scriptVersionId=189492982" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [67]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e25/sample_submission.csv
/kaggle/input/playground-series-s3e25/train.csv
/kaggle/input/playground-series-s3e25/test.csv


In [68]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import median_absolute_error
import lightgbm as lgb
import joblib
import matplotlib.pyplot as plt
import numpy as np
import optuna

In [69]:
train_data = pd.read_csv('/kaggle/input/playground-series-s3e25/train.csv')
test_data = pd.read_csv('/kaggle/input/playground-series-s3e25/test.csv')

In [70]:
train_data = train_data.drop(columns=['id'])

In [71]:
y = train_data['Hardness']
X = train_data.drop(columns='Hardness')

In [72]:
X_train, X_val, y_train, y_val = train_test_split(X, y, 
                                                  test_size=0.2, 
                                                  random_state=42)

In [73]:
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

In [74]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

In [75]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [76]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [77]:
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_val_preprocessed = preprocessor.transform(X_val)

In [78]:
def objective(trial):
    param = {
        'num_leaves': trial.suggest_int('num_leaves', 20, 30),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 1e-1),
        'n_estimators': trial.suggest_int('n_estimators', 30, 50),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 20)
    }
    
    model = lgb.LGBMRegressor(**param)
    model.fit(X_train_preprocessed, y_train)
    y_val_pred = model.predict(X_val_preprocessed)
    val_score = median_absolute_error(y_val, y_val_pred)
    return val_score

In [79]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

[I 2024-07-23 18:53:43,561] A new study created in memory with name: no-name-c12796e7-1932-4c6d-81ca-1763ebf7d384
/tmp/ipykernel_33/1682860692.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 1e-1),
[I 2024-07-23 18:53:43,687] Trial 0 finished with value: 1.0046813292264356 and parameters: {'num_leaves': 22, 'learning_rate': 0.02339572985471249, 'n_estimators': 37, 'min_child_samples': 19}. Best is trial 0 with value: 1.0046813292264356.
/tmp/ipykernel_33/1682860692.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 1e-1),


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001798 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2553
[LightGBM] [Info] Number of data points in the train set: 8325, number of used features: 11
[LightGBM] [Info] Start training from score 4.650150
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001783 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2553
[LightGBM] [Info] Number of data points in the train set: 8325, number of used features: 11
[LightGBM] [Info] Start training from score 4.650150


[I 2024-07-23 18:53:43,835] Trial 1 finished with value: 1.322754732629174 and parameters: {'num_leaves': 21, 'learning_rate': 0.0006178995215687864, 'n_estimators': 44, 'min_child_samples': 17}. Best is trial 0 with value: 1.0046813292264356.
/tmp/ipykernel_33/1682860692.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 1e-1),
[I 2024-07-23 18:53:43,968] Trial 2 finished with value: 1.3135886945571844 and parameters: {'num_leaves': 26, 'learning_rate': 0.0010527267989081403, 'n_estimators': 34, 'min_child_samples': 6}. Best is trial 0 with value: 1.0046813292264356.
/tmp/ipykernel_33/1682860692.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. 

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001796 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2553
[LightGBM] [Info] Number of data points in the train set: 8325, number of used features: 11
[LightGBM] [Info] Start training from score 4.650150
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001836 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2553
[LightGBM] [Info] Number of data points in the train set: 8325, number of used features: 11
[LightGBM] [Info] Start training from score 4.650150


[I 2024-07-23 18:53:44,083] Trial 3 finished with value: 1.2678396091454385 and parameters: {'num_leaves': 24, 'learning_rate': 0.002619879257452361, 'n_estimators': 32, 'min_child_samples': 18}. Best is trial 0 with value: 1.0046813292264356.
/tmp/ipykernel_33/1682860692.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 1e-1),
[I 2024-07-23 18:53:44,211] Trial 4 finished with value: 0.7842501527641037 and parameters: {'num_leaves': 22, 'learning_rate': 0.05363367365205888, 'n_estimators': 37, 'min_child_samples': 15}. Best is trial 4 with value: 0.7842501527641037.
/tmp/ipykernel_33/1682860692.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. U

...

[I 2024-07-23 18:53:51,238] Trial 45 finished with value: 0.9299914686769983 and parameters: {'num_leaves': 20, 'learning_rate': 0.031246358957501195, 'n_estimators': 35, 'min_child_samples': 12}. Best is trial 22 with value: 0.6713366299663958.
/tmp/ipykernel_33/1682860692.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 1e-1),
[I 2024-07-23 18:53:51,408] Trial 46 finished with value: 0.6916882252573877 and parameters: {'num_leaves': 21, 'learning_rate': 0.07171119560576784, 'n_estimators': 48, 'min_child_samples': 8}. Best is trial 22 with value: 0.6713366299663958.
/tmp/ipykernel_33/1682860692.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001755 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2553
[LightGBM] [Info] Number of data points in the train set: 8325, number of used features: 11
[LightGBM] [Info] Start training from score 4.650150
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001753 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2553
[LightGBM] [Info] Number of data points in the train set: 8325, number of used features: 11
[LightGBM] [Info] Start training from score 4.650150


[I 2024-07-23 18:53:51,569] Trial 47 finished with value: 1.3294592968971317 and parameters: {'num_leaves': 22, 'learning_rate': 0.00045320742067525577, 'n_estimators': 45, 'min_child_samples': 20}. Best is trial 22 with value: 0.6713366299663958.
/tmp/ipykernel_33/1682860692.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 1e-1),
[I 2024-07-23 18:53:51,740] Trial 48 finished with value: 1.0163655840424775 and parameters: {'num_leaves': 20, 'learning_rate': 0.017032088698135495, 'n_estimators': 50, 'min_child_samples': 11}. Best is trial 22 with value: 0.6713366299663958.
/tmp/ipykernel_33/1682860692.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001742 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2553
[LightGBM] [Info] Number of data points in the train set: 8325, number of used features: 11
[LightGBM] [Info] Start training from score 4.650150
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001864 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2553
[LightGBM] [Info] Number of data points in the train set: 8325, number of used features: 11
[LightGBM] [Info] Start training from score 4.650150


[I 2024-07-23 18:53:51,919] Trial 49 finished with value: 0.6874849029460537 and parameters: {'num_leaves': 26, 'learning_rate': 0.0938247596022786, 'n_estimators': 46, 'min_child_samples': 10}. Best is trial 22 with value: 0.6713366299663958.


In [80]:
best_params = study.best_params
print(f"Найкращі параметри: {best_params}")

Найкращі параметри: {'num_leaves': 23, 'learning_rate': 0.09971633555235294, 'n_estimators': 50, 'min_child_samples': 9}


In [81]:
best_model = lgb.LGBMRegressor(**best_params)
best_model.fit(X_train_preprocessed, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001759 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2553
[LightGBM] [Info] Number of data points in the train set: 8325, number of used features: 11
[LightGBM] [Info] Start training from score 4.650150


LGBMRegressor(learning_rate=0.09971633555235294, min_child_samples=9,
              n_estimators=50, num_leaves=23)

In [82]:
y_val_pred = best_model.predict(X_val_preprocessed)

In [83]:
val_median_absolute_error = median_absolute_error(y_val, y_val_pred)
print(f"Середня абсолютна помилка: {val_median_absolute_error}")

Середня абсолютна помилка: 0.6713366299663958


In [84]:
X_preprocessed = preprocessor.fit_transform(X)
best_model.fit(X_preprocessed, y)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002216 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2570
[LightGBM] [Info] Number of data points in the train set: 10407, number of used features: 11
[LightGBM] [Info] Start training from score 4.647126


LGBMRegressor(learning_rate=0.09971633555235294, min_child_samples=9,
              n_estimators=50, num_leaves=23)

In [85]:
preprocessor.fit(X)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 Index(['allelectrons_Total', 'density_Total', 'allelectrons_Average',
       'val_e_Average', 'atomicweight_Average', 'ionenergy_Average',
       'el_neg_chi_Average', 'R_vdw_element_Average', 'R_cov_element_Average',
       'zaratio_Average', 'density_Average'],
      dtype='object')),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 Index([], dtype='object'))])

In [86]:
feature_importance = best_model.feature_importances_
feature_names = numeric_features.tolist() + preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(categorical_features).tolist()
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importance
})

importance_df = importance_df.sort_values(by='Importance', ascending=False)
print(importance_df.head(10))

NotFittedError: This OneHotEncoder instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
plt.figure(figsize=(10, 8))
plt.barh(importance_df['Feature'], importance_df['Importance'])
plt.xlabel("Feature Importance")
plt.title("Feature Importance for Mineral Hardness Prediction")
plt.gca().invert_yaxis()
plt.show()

In [ ]:
id_data = test_data['id']
test_df = test_data.drop(columns=['id'])

In [ ]:
X_test_preprocessed = preprocessor.transform(test_df)

In [ ]:
y_test_pred = best_model.predict(X_test_preprocessed)

In [ ]:
submission_df = pd.DataFrame({
    'id': id_data,
    'Hardness': y_test_pred
})

submission_file_path = '/kaggle/working/submission.csv'
submission_df.to_csv(submission_file_path, index=False)

print("Submission file created at:", submission_file_path)

In [ ]:
final_model = Pipeline(steps=[('preprocessor', preprocessor),
                              ('regressor', best_model)])
joblib.dump(final_model, 'mineral_hardness_model.pkl')

In [ ]:
loaded_model = joblib.load('mineral_hardness_model.pkl')

In [ ]:
example_data = test_df.iloc[:5, :]
example_data_preprocessed = preprocessor.transform(example_data)
example_pred = loaded_model.predict(example_data_preprocessed)

print("Predictions:", example_pred)

In [ ]:
y_train_pred = best_model.predict(X_preprocessed)
train_mae = median_absolute_error(y, y_train_pred)
test_mae = median_absolute_error(y_val, y_val_pred)

metrics_df = pd.DataFrame({
    'Train': [train_mae],
    'Test': [test_mae]
}, index=['Median Absolute Error'])

metrics_df.plot(kind='bar', figsize=(8, 6))
plt.title("Train vs Test Metrics")
plt.ylabel("Median Absolute Error")
plt.show()